This notebook presents how to use the renovexpy package to:
- simulate a building with EnergyPlus
- simulate different heating systems (based on EnergyPlus output)
- simulate PV generation (based on EnergyPlus output)
- Obtain KPIs for the building

<mark>Important: when using the package for your own work, always put your scripts in a different folder than "renovexpy". The files in the package shouldn't be modified unless you want to add/modify a feature on purpose. For instance, make a copy of this notebook elsewhere.</mark>

In [30]:
# Import packages
from pathlib import Path
from renovexpy import __file__ as renovexpy_file
from renovexpy.energyplus import run_eplus_simulation
from renovexpy.heating_system import simulate_heating_systems
from renovexpy.pv import simulate_pv_generation
from renovexpy.kpi import get_simulated_KPIs

renovexpy_dir = Path(renovexpy_file).parent
sim_dir = renovexpy_dir / "examples/simulation/test_CO2_R_wall_low"

In [31]:
run_eplus_simulation(
    building_type="terraced_house",
    building_position="corner",
    building_orientation="S",
    floor_type="Wood",
    floor_insulation=None,
    roof_insulation=None,
    wall_insulation=["Rockwool", "Cavity", 6],
    glazing="DoubleGlz",
    window_frame="PVC",
    WWR=0.4,
    airtightness=0.4,
    n_occupants=4,
    occupant_activity_level="mid",
    heated_zones=["0F", "1FN", "1FS"],
    heating_setpoint="Always_21",
    vent_type="A",
    window_vent_profile=3,
    use_vent_grilles=False,
    mech_vent_profile=1,
    lighting_power_per_area=1,
    equipment_power_per_area=1,
    shaded_surfaces=[],
    shading_position="External",
    shading_profile=1,
    sim_dir=sim_dir,
    epw_file="DeBilt_2000"
)

In [32]:
simulate_heating_systems(
    sim_dir=sim_dir,
    radiator_areas=[4, 5, 6],
    DHW_profile="2P Mid",
)
simulate_pv_generation(sim_dir, N_panels_frac=[0, 0.5, 1])

In [33]:
df_kpi = get_simulated_KPIs(sim_dir)
display(df_kpi.round(2))

,Heating demand [kWh],Gas consumption [kWh],Electricity consumption [kWh],Electricity OPP [kW],Overheating [h],CO2 excess [h],CO2 emissions [kgCO2],Operational cost [€],heating_system,radiator_area,N_pv
0,6394.46,8355.13,1058.09,0.26,1320.0,7665,17973.81,1500.05,HR107,4.0,0
1,6394.46,8355.13,512.18,0.25,1320.0,7665,17688.31,1149.92,HR107,4.0,11
2,6394.46,8355.13,481.35,0.25,1320.0,7665,17672.18,1001.58,HR107,4.0,22
3,6394.46,9385.16,1058.09,0.26,1320.0,7665,20121.44,1643.94,VR,4.0,0
4,6394.46,9385.16,512.18,0.25,1320.0,7665,19835.94,1293.80,VR,4.0,11
...,...,...,...,...,...,...,...,...,...,...,...
76,6394.46,0.13,1797.83,1.07,1320.0,7665,940.53,405.73,HP 3kW Intergas + HR107 Parallel,6.0,11
77,6394.46,0.13,1699.93,1.05,1320.0,7665,889.33,238.57,HP 3kW Intergas + HR107 Parallel,6.0,22
78,6394.46,62.35,2599.02,1.16,1320.0,7665,1489.28,834.99,HP 3kW Intergas + HR107 Alternate,6.0,0
79,6394.46,62.35,1783.55,1.03,1320.0,7665,1062.79,409.86,HP 3kW Intergas + HR107 Alternate,6.0,11
